# Moving Average Crossover Trading Strategy

## How It Works

The strategy utilizes two moving averages:

- **Short-term Moving Average ($MA_{\text{short}}$)**: calculated over a smaller window $n_s$ (e.g., 50 days).
- **Long-term Moving Average ($MA_{\text{long}}$)**: calculated over a larger window $n_l > n_s$ (e.g., 200 days).

Formally:
$$
MA_{\text{short}}(t) = \frac{1}{n_s} \sum_{k=0}^{n_s-1} P_{t-k}
$$
$$
MA_{\text{long}}(t) = \frac{1}{n_l} \sum_{k=0}^{n_l-1} P_{t-k}
$$
Where $P_t$ is the closing price at time $t$.

## Trading Signals

- **Buy Signal ("Golden Cross")**: when $MA_{\text{short}}(t)$ crosses above $MA_{\text{long}}(t)$.
- **Sell Signal ("Death Cross")**: when $MA_{\text{short}}(t)$ crosses below $MA_{\text{long}}(t)$.

### Signal Logic in Math

$$
\begin{align*}
\text{Buy Signal:} \quad & MA_{\text{short}}(t-1) \leq MA_{\text{long}}(t-1) \quad \text{and} \quad MA_{\text{short}}(t) > MA_{\text{long}}(t) \\
\text{Sell Signal:} \quad & MA_{\text{short}}(t-1) \geq MA_{\text{long}}(t-1) \quad \text{and} \quad MA_{\text{short}}(t) < MA_{\text{long}}(t)
\end{align*}
$$

## Example Log Decision

Every time a crossover event is detected, print:






Where the signal is either "Golden Cross" (Buy) or "Death Cross" (Sell).

## Considerations

- The window sizes ($n_s$, $n_l$) influence signal frequency and sensitivity.
- Larger $n_l$ smooth out noise but delay signals; smaller $n_s$ reacts faster but can cause false signals.
- Detailed logs for each decision are crucial for debugging and backtesting.
- Works the same on Mac, Windows, Linux; only package installation differs.



![Moving Average Crossover Example](macexample.png)

https://www.youtube.com/watch?v=IvvUbh-cnX4

In [3]:
%matplotlib inline

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Parameters
ticker = "SOXL"
short_window = 50
long_window = 200

# Download historical data (último año)
data = yf.download(ticker, period="1y", interval="1d", auto_adjust=True, progress=False)

if data.empty:
    print(f"No data found for {ticker}. Please check the ticker or data source.")
else:
    print(f"Data for {ticker} downloaded with {len(data)} rows.")

    # Calculate Exponential Moving Averages (EMA)
    data['Short_EMA'] = data['Close'].ewm(span=short_window, adjust=False).mean()
    data['Long_EMA'] = data['Close'].ewm(span=long_window, adjust=False).mean()

    in_position = False
    trades = []

    print("Date       | Close   | Short_EMA | Long_EMA | Signal       | Action")
    print("---------------------------------------------------------------")

    for i in range(1, len(data)):
        row = data.iloc[i]
        prev = data.iloc[i - 1]
        date = row.name.date()
        short_ema, long_ema = row['Short_EMA'], row['Long_EMA']
        prev_short_ema, prev_long_ema = prev['Short_EMA'], prev['Long_EMA']
        action = ""

        if pd.notna(short_ema) and pd.notna(long_ema):
            # Golden Cross
            if short_ema > long_ema and prev_short_ema <= prev_long_ema:
                action = "BUY"
                in_position = True
                print(f"{date} | {row['Close']:.2f} | {short_ema:.2f} | {long_ema:.2f} | Golden Cross | {action}")
                trades.append({'date': date, 'type': 'BUY', 'price': row['Close']})
            # Death Cross
            elif short_ema < long_ema and prev_short_ema >= prev_long_ema:
                if in_position:
                    action = "SELL"
                    in_position = False
                    print(f"{date} | {row['Close']:.2f} | {short_ema:.2f} | {long_ema:.2f} | Death Cross  | {action}")
                    trades.append({'date': date, 'type': 'SELL', 'price': row['Close']})

    print("\nTrade Log:")
    for t in trades:
        print(f"{t['date']} - {t['type']} at {t['price']}")

    # Plot results
    plt.figure(figsize=(14, 7))
    plt.plot(data['Close'], label='Close Price')
    plt.plot(data['Short_EMA'], label=f'{short_window}-day EMA')
    plt.plot(data['Long_EMA'], label=f'{long_window}-day EMA')
    plt.title(f"{ticker} Price and Exponential Moving Averages")
    plt.legend()
    plt.grid(True)
    plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [11]:
import yfinance as yf

data = yf.download("SOXL", period="5d", interval="1d", auto_adjust=True)
print(data)


[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low   Open     Volume
Ticker           SOXL       SOXL       SOXL   SOXL       SOXL
Date                                                         
2025-08-29  26.040001  27.350000  25.670000  27.32  108411300
2025-09-02  25.250000  25.280001  23.799999  24.08  101906700
2025-09-03  24.910000  25.410000  24.120001  25.41   79482200
2025-09-04  25.650000  25.730000  23.980000  24.41   81641200
2025-09-05  26.450001  26.969999  25.600000  26.82  110467500
